In [1]:
from utils.experimentor_base import *
from utils.experimentor_proteins import * 
from utils.constants import *

In [2]:
config = {
    "dataset_name": Dataset.OGBN_PROTEINS,
    "model_type": ModelType.GATV1,
    'num_of_epochs': 1,
    'num_of_runs': 1,
    'patience_period': 30,
    
    'batch_size': 256,
    'test_batch_size': 256,
    'num_workers': 2,
    'force_cpu': False,
    'test_frequency': 1,
    'console_log_freq': 1,
    'do_train_tqdm_logging': False,
    
    'lr': 0.001,
    'num_of_layers': 2, 
    'num_heads': 1,
    'hidden_size': 128,
    'dropout': 0.5,  
    "use_layer_norm": False,
    "use_batch_norm": False,
    
    'nbor_degree': 1,
    'adj_mode': AdjacencyMode.OneStep,
    'sparse': False
}

In [3]:
exp = Experimentor_Proteins if config["dataset_name"] == Dataset.OGBN_PROTEINS else Experimentor
experimentor = exp(config)
experimentor.run()

Run 01:


Epoch 01: 100%|██████████| 90941/90941 [00:18<00:00, 4797.28it/s] 


Epoch 01| Loss: 2.2085| Acc: 0.4214| Train Time: 18.9601s


Evaluating: 100%|██████████| 338686/338686 [00:14<00:00, 23251.38it/s]

Train: 0.5602| Val: 0.5659| Test: 0.5212| Eval Time: 14.5992s

Result of  1. run| Train: 0.5602| Val: 0.5659| Test: 0.5212

